# 課題5：映画レビューの評判分析

本課題ではAmazon傘下の「IMDb」に投稿された映画のレビュー（英語）を分析し、レビューがPositive（ポジティブ）か、Negative（ネガティブ）かの判別を行ないます。

データセットは、以下のサイトで配布されているものを利用します。

[Large Movie Review Dataset](https://ai.stanford.edu/%7Eamaas/data/sentiment/)

わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なライブラリのimport

In [36]:
# （変更しないでください）

# 必要なライブラリのimport
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 文章ファイル検索用
import glob
import collections
from sklearn.feature_extraction import DictVectorizer

# DataFrameですべての列を表示する設定
pd.options.display.max_columns = None

# seabornによる装飾を適用する
sns.set_theme()

## 2. データの読み込み

In [11]:
# ダウンロードした圧縮ファイルを解凍する（変更しないでください）
#!tar zxvf aclImdb_v1.tar.gz

#時間がかかりすぎたため、解凍したフォルダをアップロードしました

*./aclImdb* フォルダ内にあるファイルを読み込みます。

In [39]:
# trainフォルダのファイル一覧を取得（変更しないでください）
train_neg_files = glob.glob("./aclImdb/train/neg/*")
train_pos_files = glob.glob("./aclImdb/train/pos/*")

# testフォルダのファイル一覧を取得（変更しないでください）
test_neg_files = glob.glob("./aclImdb/test/neg/*")
test_pos_files = glob.glob("./aclImdb/test/pos/*")

In [40]:
# それぞれのファイル数を確認
print(len(train_neg_files))
print(len(train_pos_files))
print(len(test_neg_files))
print(len(test_pos_files))

12500
12500
12500
12500


前処理をするため、合計50000あるファイルをリストにまとめます。

In [41]:
# ファイル名をまとめたリストを用意（変更しないでください）
filenames = train_neg_files + train_pos_files + test_neg_files + test_pos_files

# filenamesの長さを確認（変更しないでください）
len(filenames)

50000

リストの最初と最後のファイルを確認してみます。

In [42]:
# エンコーディング用定数（変更しないでください）
ENCODING = 'utf-8'

In [43]:
# 最初のファイルの内容を確認
with open(filenames[0], encoding=ENCODING) as f:
    text = f.read()
    print(text)

How they got Al Pacino to play in this movie is beyond me. This movie is absolutely terrible. I discovered, after reading some of the other reviews, that a couple of people actually enjoyed this film, which deeply puzzles me, because I do not see how anyone in their right mind could possibly enjoy a movie as awful as Revolution. It's not just that it's a bad movie, with a lame plot and overall strangeness that is extremely unpleasant, but it seems as if the filmmakers were either mentally retarded (which is a very possible explanation as to why this movie sucks like it does, though it probably still sucks even compared to other films made by retards) or deliberately made every illogical decision to make this movie suck as much as possible. For example, we see Donald Sutherland running around with a huge, fat ugly mole on his face. He does not normally have a mole. The mole does not add to his character. It is extremely ugly and distracting. It's not like Robert De Niro's mole; it's muc

In [44]:
# 最後のファイルの内容を確認
with open(filenames[-1], encoding=ENCODING) as f:
    text = f.read()
    print(text)

This is the last film of a trilogy by the brilliant Turkish director, Nuri Bilge Ceylan, whose last film Mayis Sikintisi -which was very Cehovian- was shown in prestigious film festivals. Differing from his previous films, the story of 'Uzak' is set on Istanbul which is one of the most crowded cities of the world. However, in Ceylan's film, we do see only minor traces of that huge crowd. Rather he choose to focus on two characters, one photographer and one of his relatives who comes from his small village to find a job on transatlantic ships. The photographer, who -we understand that- has also immigrated to the city, seems to be inhabited the customs of the city life, not only in material sense. In his relation to his relative, we see him first as caring and tolerant, however, when he could not find a job, our suburbian character starts to be disturbed for sharing his private 'space' with someone whose leaving date becomes ambiguous. I will not reveal the tactics he develops in order t

## 3. データの前処理

データの前処理として、形態素解析と行列への変換を行ないます。

### 形態素解析

In [45]:
# 文字列の中で使われている単語ごとの数を返す関数を作成
#（レッスン本編の内容を確認して、下記にコードを追記してください）
def get_word_count(text, min_length=3):
    #ノイズ除去
    for ch in ".,:;!?-+*/=()[]{}<>~^#$@%&'\"_0123456789":
        text = text.replace(ch, ' ')
    
    #単語に分割
    _words = text.strip().split()

    #表記ゆれ補正
    _words = [word.lower() for word in _words if len(word) >= min_length]
    
    # collections.Counterの戻り値は辞書型のサブクラス
    _count = collections.Counter(_words)

    # 辞書型に変換して返す
    return dict(_count)

In [46]:
# 最初のファイルを使って、先ほど作成した関数をテスト
with open(filenames[0], 'r', encoding=ENCODING) as f:
    text = f.read()

    get_word_count(text)

In [47]:
# 単語ごとの数のリストを作成（変更しないでください）
word_count_data = []

In [48]:
# すべてのファイルに対して、先ほど作成した関数を実行
for filename in filenames:
    with open(filename, 'r', encoding=ENCODING) as f:
        text = f.read()
        count = get_word_count(text)
        word_count_data.append(count)

In [49]:
# 単語ごとの数のリストの長さを確認
len(word_count_data)

50000

In [50]:
# 単語ごとの数のリストの0番目を表示
word_count_data[0]

{'how': 5,
 'they': 5,
 'got': 3,
 'pacino': 1,
 'play': 4,
 'this': 7,
 'movie': 13,
 'beyond': 1,
 'absolutely': 1,
 'terrible': 4,
 'discovered': 1,
 'after': 1,
 'reading': 1,
 'some': 1,
 'the': 31,
 'other': 2,
 'reviews': 1,
 'that': 9,
 'couple': 1,
 'people': 1,
 'actually': 1,
 'enjoyed': 1,
 'film': 1,
 'which': 4,
 'deeply': 1,
 'puzzles': 1,
 'because': 3,
 'not': 7,
 'see': 6,
 'anyone': 1,
 'their': 2,
 'right': 2,
 'mind': 1,
 'could': 2,
 'possibly': 2,
 'enjoy': 1,
 'awful': 1,
 'revolution': 1,
 'just': 7,
 'bad': 3,
 'with': 2,
 'lame': 1,
 'plot': 1,
 'and': 15,
 'overall': 1,
 'strangeness': 1,
 'extremely': 2,
 'unpleasant': 1,
 'but': 2,
 'seems': 1,
 'filmmakers': 5,
 'were': 2,
 'either': 1,
 'mentally': 1,
 'retarded': 1,
 'very': 2,
 'possible': 2,
 'explanation': 1,
 'why': 2,
 'sucks': 4,
 'like': 5,
 'does': 5,
 'though': 1,
 'probably': 1,
 'still': 1,
 'even': 4,
 'compared': 1,
 'films': 1,
 'made': 2,
 'retards': 1,
 'deliberately': 1,
 'every': 1,
 '

### 行列への変換

In [54]:
# DictVectorizerを使用して行列に変換し、datasetに格納する
vec = DictVectorizer()
dataset = vec.fit_transform(word_count_data)


In [55]:
# datasetの大きさを確認
dataset.shape

(50000, 101249)

In [56]:
# 各列に対応した単語を取得
vec.get_feature_names_out()

array(['\x08\x08\x08\x08a', '\x10own', '\\and\\', ..., '…although',
       '…but', '…until'], dtype=object)

## 4. 機械学習の実施

In [57]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

目的変数と説明変数を用意します。

In [58]:
# 目的変数Yの用意
# neg12500 + pos12500 + neg12500 + pos12500 = 50000
Y = np.array([0]*12500 + [1]*12500 + [0]*12500 + [1]*12500)

In [59]:
# 上記のY、および前処理されたdatasetからデータを分割し、
# X_train, Y_train, X_test, Y_testに格納する
#
# 詳細：
#   - dataset の先頭から25000件を 変数 X_train に、残りを変数 X_test に代入
#   - 目的変数 Y の先頭から25000件を 変数 Y_train に、残りを Y_test に代入

X_train = dataset[:25000]
X_test = dataset[25000:]
Y_train = Y[:25000]
Y_test = Y[25000:]


In [60]:
# X_trainとY_trainを、train_test_splitで7:3に分割し、3割のほうを検証データ（X_valid, Y_valid）にする
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)


In [63]:
# ロジスティック回帰モデルを作成し、学習して、検証データによる予測を実施する
Logistic_model = LogisticRegression(max_iter=500)
Logistic_model.fit(X_train, Y_train)
Y_pred = Logistic_model.predict(X_valid)

# classification_reportを実行し、検証データによるモデルの評価を行なう
print(classification_report(Y_valid, Y_pred))

              precision    recall  f1-score   support

           0       0.88      0.85      0.86      2609
           1       0.86      0.88      0.87      2641

    accuracy                           0.87      5250
   macro avg       0.87      0.87      0.87      5250
weighted avg       0.87      0.87      0.87      5250



## 5. テストデータによる評価

最後に、テストデータで評価を行ないましょう。

In [64]:
# テストデータで予測を実施する
Y_pred = Logistic_model.predict(X_test)

# classification_reportを実行し、テストデータによるモデルの評価を行なう
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.87      0.86      0.87      3767
           1       0.86      0.87      0.87      3733

    accuracy                           0.87      7500
   macro avg       0.87      0.87      0.87      7500
weighted avg       0.87      0.87      0.87      7500

